<a href="https://colab.research.google.com/github/jerryw18/job_hack/blob/main/JW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install uv
# !uv pip install --system -qU llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai llama-index-llms-openai-like "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv


In [ ]:
from os import environ from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4")

In [ ]:
import phoenix as px
session = px.launch_app()

In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

In [ ]:
# load data
user0_docs = SimpleDirectoryReader(
    input_files=["./dataset/user0/"]
    ).load_data()

# build index
user_docs_index = VectorStoreIndex.from_documents(user0_docs, show_progress=True)


# persist index, JWTODO, do this later
#lyft_index.storage_context.persist(persist_dir="./storage/lyft")
#uber_index.storage_context.persist(persist_dir="./storage/uber")

In [ ]:
user0_engine = user0_docs_index.as_query_engine(similarity_top_k=3, llm=llm)

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=user0_engine,
        metadata=ToolMetadata(
            name="user0_cover_letter",
            description=(
                "build a cover letter from user profile and job position."
            ),
        ),
    ),
]

In [ ]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

In [ ]:
response = agent.chat("Could you build a cover letter that presents a strong case on why the applicant is a good candidate for the position?")
print(str(response))